# Chinese_Poems_Generation_with_Deep_Neural_Networks

## Main tasks

1. Preprocessing for language modeling data
    * Clean noise
    * Create a dictionary that maps words to unique IDs
    * Convert words to ID
    * Reshape sequences to unified lengths
2. Constructed a seq2seq model using RNNetwork with LSTM
    * Hyperparameters
    * Define a training operation for an epoch
    * Define an operation for printing test data
    * Define training controller
3. Training and evaluation
    * Observe loss
    * Evaluate on test set

## Task: Language Modeling
Constructed a sequence-to-sequence model using Recurrent Neural Network with Long
Short-term Memory cells

### Definition
We input a sequence of words/characters to an RNN so that it can learn the probability distribution of the next word/character in the sequence given the history of previous characters. This will then allow us to generate text one unit at a time.

We will use Full Tang poetry(全唐诗) as our training data, and try to generate new poems later!

## 1. Preprocessing for language modeling data

### Clean noise

There are all kinds of noise in text data. We need to check the correctness of the preprocessing!

The format of our input data is like this:

`(optional title + ":")poem`

We will use only the poem part and not the title.

However, some special cases like:

* 河鱼未上冻，江蛰已闻雷。（见《纬略》）
* □□□□□

We need some preprocessing.

* Remove title
* Remove spaces
* Remove empty symbols
* Replace other symbols

Finally, we will randomize them.

In [1]:
import re
import numpy as np

data_filename ='poetry.txt'
poems = []
with open(data_filename, "r") as in_file:
  for line in in_file.readlines():
    line = line.strip()
    # find title if exists
    if ':' in line:
      line = line.split(':')
    # some poems are empty
    if len(line) == 2:
      poem = line[1]
    else:
      continue
    # discard if contains special symbols
    if re.search(r'[(（《_□]', poem):
      continue
    # discard if too short or too long
    if len(poem) < 5 or len(poem) > 40:
      continue
    # remove symbols
    poem = re.sub(u'[，。]','',poem)
    poems.append(poem)

poems = np.random.permutation(poems)

We select 5 poems as our test set.

In [2]:
poems_train, poems_test = poems[:-5], poems[-5:]
len(poems_train), len(poems_test)

(11098, 5)

### Word to ID

As mentioned lst time, we can use the tokenizer in Keras to help us. We set
```python
Tokenizer(num_words=None, lower=False, char_level=True)
```
to not limit the number of words in the dictionary, and use character as our unit.

We also need to correct the dictionary because it starts from 1 and that will be a problem later!

In [3]:
import time
# from tensorflow.keras.preprocessing.text import Tokenizer
from keras.preprocessing.text import Tokenizer

poem_tokenizer = Tokenizer(num_words=None, lower=False, char_level=True)
# Create word to ID dictionary
poem_tokenizer.fit_on_texts(poems)
# Get dictionary
word_index = poem_tokenizer.word_index

# Note that ID starts from 1!!
# We need to add special ID 0
word_index["<PAD>"] = 0
# Create ID to word 
reverse_word_index = dict([(v, k) for (k, v) in word_index.items()])
print("Number of unique chars: {}".format(len(word_index)))

Using TensorFlow backend.


Number of unique chars: 4762


Again, always check if there is any strange symbols in the dictionary. Here we only print first and last parts.

In [4]:
# sort word index by ID
for (w,i) in sorted(word_index.items(), key=lambda w: w[1]):
# print some words to check if there are errors!
  if i > 10 and i < len(word_index)-5: continue
  print("{} {}".format(w,i))

<PAD> 0
不 1
人 2
一 3
山 4
风 5
无 6
花 7
来 8
日 9
春 10
诡 4757
邢 4758
颉 4759
颃 4760
荍 4761


In [5]:
# Apply word to ID on training and test set
poems_train = poem_tokenizer.texts_to_sequences(poems_train)
poems_test = poem_tokenizer.texts_to_sequences(poems_test)
# Check and see if there is any error
print(poems_train[0])
print(''.join([reverse_word_index[w] for w in poems_train[0]]))

[154, 45, 69, 194, 89, 16, 61, 58, 83, 4, 7, 68, 135, 223, 21, 73, 625, 211, 33, 235, 133, 32, 425, 5, 13, 486, 29, 210]
两行客泪愁中落万树山花雨后残君到扬州见桃叶为传风水渡江难


### Prepare the data for input

We flatten the input to a long list.

In [6]:
# flatten to a long string of characters
poems_train = [w for po in poems_train for w in po]

# flatten to a long string of characters
poems_test = [w for po in poems_test for w in po]

### Define an input object

We need to put the input into batches.
* Reshape input data into a rectangular matrix and crop remainders
* Calculate shape of each batch
* Generate batch with input and output = input shift by one time step


In [7]:
import tensorflow as tf

In [8]:
class PoemInput(object):
  def __init__(self, data, config, name=None):
    self.batch_size = batch_size = config.batch_size
    self.num_steps = num_steps = config.num_steps
    self.epoch_size = ((len(data) // batch_size) - 1) // num_steps
    self.sources, self.targets = self.input_producer(
        data, batch_size, num_steps, name=name)

  def input_producer(self, raw_data, batch_size, num_steps, name=None):
    """Reshape the poem data to form input and output.
    This chunks the raw_data into batches of examples and returns Tensors that
    are drawn from these batches.
    Args:
      raw_data: a list of words
      batch_size: int, the batch size.
      num_steps: int, the sequence length.
      name: the name of this operation (optional).
    Returns:
      A pair of Tensors, each shaped [batch_size, num_steps]. The second element
      of the tuple is the same data time-shifted to the right by one.
    """
    raw_data = tf.convert_to_tensor(raw_data, name="raw_data", dtype=tf.int32)
    # get size of the 1-d tensor
    data_len = tf.size(raw_data)
    # calculate how many batches
    batch_len = data_len // batch_size
    # crop data that does not fit in a batch
    data = tf.reshape(raw_data[0:batch_size*batch_len],
                      [batch_size, batch_len])
    # calculate how many batches in an epoch
    epoch_size = (batch_len - 1) // num_steps
    # make sure there is at least one batch
    assertion = tf.assert_positive(epoch_size,
        message="epoch_size == 0, decrease batch_size or num_steps")
    with tf.control_dependencies([assertion]):
      epoch_size = tf.identity(epoch_size, name="epoch_size")

    # start generating slices
    # range_input_producer returns a sequence of IDs 
    i = tf.train.range_input_producer(epoch_size, shuffle=False).dequeue()
    x = data[:, i*num_steps  :(i+1)*num_steps]
    y = data[:, i*num_steps+1:(i+1)*num_steps+1]
    return x, y

## 2. Constructed a seq2seq model using RNNetwork with LSTM 

### Define hyperparameters

In [9]:
# Define hyperparameters
class Hparam(object):
  learning_rate = 1.0
  max_grad_norm = 5
  num_layers = 2
  num_steps = 10
  vocab_size = len(word_index)
  embedding_size = 100
  hidden_size = 100
  warmup_epochs = 2
  num_epochs_to_train = 40
  keep_prob = 0.8
  lr_decay = 0.95
  batch_size = 100

config = Hparam()

### Construct model
In this step, the entire model structure must be defined completely. Including
* Input
* Size of layers
* Connection between layers
* Variables in layers
* Output
* Loss
* Operations that apply the gradients (optimizer)
* Placeholder for feeding special values
* Properties that can be read from outside

Note that we will use CudnnLSTM to speed up our training if available. However, I will provide two versions of LSTM here in case you cannot find a machine with GPUs.

In [10]:
from tensorflow.contrib.cudnn_rnn import CudnnLSTM
from tensorflow.contrib.rnn import BasicLSTMCell, MultiRNNCell
# from tensorflow.nn import embedding_lookup, dropout
from keras.layers import Dropout

# Build our model
class MyModel(object):
  def __init__(self, is_training, config, input_):
    self._is_training = is_training
    self._input = input_
    self._rnn_params = None
    self._cell = None
    self.batch_size = input_.batch_size
    self.num_steps = input_.num_steps
    rnn_size = config.hidden_size
    vocab_size = config.vocab_size
    embedding_size = config.embedding_size

    # Embeddings can only exist on CPU
    with tf.device("/cpu:0"):
      embedding_weights = tf.get_variable("embedding", \
                     [vocab_size, embedding_size])
      embed_inputs = tf.nn.embedding_lookup(embedding_weights, input_.sources)

    if is_training and config.keep_prob < 1.:
      embed_inputs = tf.nn.dropout(embed_inputs, config.keep_prob)

    # build RNN using CudnnLSTM
    # output, _ = self._build_rnn(embed_inputs, config, is_training)
    # build RNN using basic LSTM
    output, _ = self._build_rnn_old_lstm(embed_inputs, config, is_training)

    # Remember RNN output is [batch_size x time, rnnsize]
    # Dense layer for projecting onto vocabulary size
    softmax_w = tf.get_variable("softmax_w", [rnn_size, vocab_size])
    softmax_b = tf.get_variable("softmax_b", [vocab_size])
    logits = tf.nn.xw_plus_b(output, softmax_w, softmax_b)
    # Reshape logits to be a 3-D tensor for sequence loss
    logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
    self._logits = logits

    # Use the contrib sequence loss and average over the batches
    loss = tf.contrib.seq2seq.sequence_loss(
        logits,
        input_.targets,
        tf.ones([self.batch_size, self.num_steps]),
        average_across_timesteps=False,
        average_across_batch=True)

    # Update the cost
    self._cost = tf.reduce_sum(loss)

    if not is_training:
      return

    # A variable to store learning rate
    self._lr = tf.Variable(0.0, trainable=False)

    # Calculate gradients
    # Get a list of trainable variables
    tvars = tf.trainable_variables()
    # Get gradient and clip by norm
    grads, _ = tf.clip_by_global_norm(\
                 tf.gradients(self._cost, tvars),
                 config.max_grad_norm)
    # Define an optimizer
    # Note that the optimizer reads the value of learning rate from variable
    optimizer = tf.train.GradientDescentOptimizer(self._lr)
    # Define an operation that actually applies the gradients
    self._train_op = optimizer.apply_gradients(
        zip(grads, tvars),
        global_step=tf.train.get_or_create_global_step())
    # A placeholder for feeding new learning rates
    self._new_lr = tf.placeholder(
         tf.float32, shape=[], name="new_learning_rate")
    self._lr_update_op = tf.assign(self._lr, self._new_lr)

# build RNN using CudnnLSTM    
  def _build_rnn(self, inputs, config, is_training):
    # RNN requires time-major
    inputs = tf.transpose(inputs, [1, 0, 2])
    self._cell = CudnnLSTM(
        num_layers=config.num_layers,
        num_units=config.hidden_size,
        )
    self._cell.build(inputs.get_shape())
    outputs, state = self._cell(inputs)
    # Transpose from time-major to batch-major
    outputs = tf.transpose(outputs, [1, 0, 2])
    # Reshape from [batch, time, rnnsize] to [batch x time, rnnsize]
    # For computing softmax later
    outputs = tf.reshape(outputs, [-1, config.hidden_size])
    return outputs, state

# build RNN using basic LSTM
  def _build_rnn_old_lstm(self, inputs, config, is_training):
    def make_cell():
      cell = BasicLSTMCell(
        config.hidden_size, forget_bias=0.0, state_is_tuple=True,
        reuse=not is_training)
      if is_training and config.keep_prob < 1:
        cell = tf.contrib.rnn.DropoutWrapper(
            cell, output_keep_prob=config.keep_prob)
      return cell

    cell = tf.contrib.rnn.MultiRNNCell(
        [make_cell() for _ in range(config.num_layers)], state_is_tuple=True)

    self._initial_state = cell.zero_state(config.batch_size, tf.float32)
    state = self._initial_state
    outputs = []
    inputs = tf.unstack(inputs, num=self.num_steps, axis=1)
    outputs, state = tf.nn.static_rnn(cell, inputs,
                                      initial_state=self._initial_state)
    output = tf.reshape(tf.concat(outputs, 1), [-1, config.hidden_size])
    return output, state
  
  def assign_lr(self, session, lr_value):
    session.run(self._lr_update_op, feed_dict={self._new_lr: lr_value})

  @property
  def input(self):
    return self._input

  @property
  def cost(self):
    return self._cost

  @property
  def lr(self):
    return self._lr

  @property
  def train_op(self):
    return self._train_op

  @property
  def logits(self):
    return self._logits

### Define a training operation for an epoch
This procedure gets the output from the model for each batch.
We need a dictionary with these keys:

* "cost": Reads the propertie `model.cost` that we defined above. 
* "do_op": Perform operation `model.train_op` that applies gradients

After running (calling `session.run()`), the same key will contain the return values.

We can add any key in the dictionary that corresponds to `@property` in the model!

In [11]:
def run_epoch(session, model, do_op=None, verbose=False):
  start_time = time.time()
  costs = 0.0
  iters = 0
  feed_to_model_dict = {
      "cost": model.cost,
  }
  # if an operation is provided, put that in the feed
  if do_op is not None:
    feed_to_model_dict["do_op"] = do_op

  for step in range(model.input.epoch_size):
    # use the session to run, feed the dictionary
    s_out = session.run(feed_to_model_dict)
    # The returned dictionary will contain the information we need
    cost = s_out["cost"]
    # Accumulate cost
    costs += cost
    # Accumulate number of training steps
    iters += model.input.num_steps
    # Print loss periodically
    if verbose and step % (model.input.epoch_size // 10) == 10:
      print("%.0f%% ppl: %.3f, speed: %.0f char/sec" %
            (step * 100.0 / model.input.epoch_size, \
             np.exp(costs/iters), \
             iters * model.input.batch_size/(time.time() - start_time)))

  return np.exp(costs / iters)


### Define an operation for printing test data
In the end, we want to see the generated poems. We will modify the decode procedure from last week's codelab to show Chinese characters.

Here, we will again use `feed_to_model_dict` to get the output from the last layer (`logits`). Since they represent probability over vocabulary, we need to calculate `argmax` to get the actual ID of the char.

In [12]:
def run_test(session, model):

  def decode_text(text):
    words = [reverse_word_index.get(i, "<UNK>") for i in text]
    fixed_width_string = []
    # limit max length = 5
    for w_pos in range(len(words)):
      fixed_width_string.append(words[w_pos])
      if (w_pos+1) % 5 == 0:
        fixed_width_string.append('\n')
    return ''.join(fixed_width_string)

  feed_to_model_dict = {
      "logits": model.logits,
  }

  vals = session.run(feed_to_model_dict)
  logits = vals["logits"]
  # Transform to a list of IDs
  logits = logits.squeeze().argmax(axis=-1).tolist()
  print(decode_text(logits))
  return

### Main training controller
Finally, we define a controller that:
* Create the model for training
* Create the model for testing, copying from the training model
* Prepare the input data
* Define what to log in the progress of training
* Create a `session` that communicates with computation graph
* Change learning rate optionally
* Get test set results


In [13]:
def main(_):

  eval_config = Hparam()
  eval_config.batch_size = 1
  eval_config.num_steps = 20

  with tf.Graph().as_default():
    initializer = tf.random_uniform_initializer(-0.1, 0.1)

    with tf.name_scope("Train"):
      # Create input producer
      train_input = PoemInput(poems_train, config, name="TrainInput")
      # Create the model instance
      with tf.variable_scope("Model", reuse=None, initializer=initializer):
        m = MyModel(is_training=True, config=config, input_=train_input)
      # Add information to logs
      tf.summary.scalar("Training_Loss", m.cost)
      tf.summary.scalar("Learning_Rate", m.lr)

    with tf.name_scope("Test"):
      # Create another input for test data
      # Note that eval_config was set locally
      test_input = PoemInput(poems_test, eval_config, name="TestInput")
      # Create another model but reuse the variables in the training model
      with tf.variable_scope("Model", reuse=True, initializer=initializer):
        mtest = MyModel(is_training=False, config=eval_config,
                         input_=test_input)
    # Hardware settings
    config_proto = tf.ConfigProto(allow_soft_placement=True)
    # Create a session that controls the training process
    # Also automatically logs and reports 
    # Note the `checkpoint_dir` setting
    with tf.train.MonitoredTrainingSession(checkpoint_dir="logs", \
                                           config=config_proto, \
                                           log_step_count_steps=-1) as session:
      for i in range(config.num_epochs_to_train):
        # Calculate learning rate decay
        lr_decay = config.lr_decay ** max(i + 1 - config.warmup_epochs, 0.0)
        # Set learning rate
        m.assign_lr(session, config.learning_rate * lr_decay)
        # Print new learning rate
        print("Epoch: %d Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        # Train one epoch and report loss
        train_perplexity = run_epoch(session, m, do_op=m.train_op,
                                     verbose=True)
        print("Epoch: %d Train Perplexity: %.3f" % (i + 1, train_perplexity))
      
      # End of training
      # Evaluate test set performance
      test_perplexity = run_epoch(session, mtest)
      print("Test Perplexity: %.3f" % test_perplexity)
      # Print some examples from test
      run_test(session, mtest)


## 3. Training and evaluation

### Start training
We can actually start training by calling the controller.

In [16]:
main(1)

INFO:tensorflow:Create CheckpointSaverHook.
INFO:tensorflow:Graph was finalized.
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflow:Saving checkpoints for 0 into logs/model.ckpt.
Epoch: 1 Learning rate: 1.000
3% ppl: 4151.129, speed: 5099 char/sec
13% ppl: 2142.248, speed: 6954 char/sec
23% ppl: 1752.580, speed: 7343 char/sec
33% ppl: 1568.996, speed: 7510 char/sec
43% ppl: 1475.089, speed: 6582 char/sec
52% ppl: 1407.886, speed: 6474 char/sec
62% ppl: 1358.670, speed: 6329 char/sec
72% ppl: 1326.180, speed: 6255 char/sec
82% ppl: 1297.161, speed: 6256 char/sec
92% ppl: 1275.113, speed: 6409 char/sec
Epoch: 1 Train Perplexity: 1263.766
Epoch: 2 Learning rate: 1.000
3% ppl: 1116.350, speed: 8352 char/sec
13% ppl: 1093.228, speed: 8356 char/sec
23% ppl: 1115.026, speed: 7806 char/sec
33% ppl: 1113.856, speed: 7241 char/sec
43% ppl: 1118.207, speed: 7041 char/sec
52% ppl: 1114.903, speed: 7077 char/sec
62% ppl: 1110.325, speed: 7089 char/sec

33% ppl: 542.683, speed: 4287 char/sec
43% ppl: 542.934, speed: 4285 char/sec
52% ppl: 539.794, speed: 4392 char/sec
62% ppl: 536.146, speed: 4513 char/sec
72% ppl: 534.139, speed: 4603 char/sec
82% ppl: 532.175, speed: 4654 char/sec
92% ppl: 530.671, speed: 4712 char/sec
Epoch: 18 Train Perplexity: 530.421
Epoch: 19 Learning rate: 0.418
3% ppl: 525.045, speed: 5378 char/sec
13% ppl: 517.536, speed: 4555 char/sec
23% ppl: 527.904, speed: 5100 char/sec
33% ppl: 527.046, speed: 5496 char/sec
43% ppl: 527.054, speed: 5585 char/sec
52% ppl: 523.982, speed: 5433 char/sec
62% ppl: 520.376, speed: 5569 char/sec
72% ppl: 518.287, speed: 5593 char/sec
82% ppl: 516.268, speed: 5612 char/sec
92% ppl: 515.350, speed: 5520 char/sec
Epoch: 19 Train Perplexity: 515.429
Epoch: 20 Learning rate: 0.397
3% ppl: 516.692, speed: 6805 char/sec
13% ppl: 507.088, speed: 5692 char/sec
23% ppl: 515.975, speed: 6244 char/sec
33% ppl: 514.220, speed: 5892 char/sec
43% ppl: 514.994, speed: 5840 char/sec
52% ppl: 5

13% ppl: 391.634, speed: 6525 char/sec
23% ppl: 397.902, speed: 6821 char/sec
33% ppl: 396.445, speed: 6484 char/sec
43% ppl: 396.905, speed: 5848 char/sec
52% ppl: 394.781, speed: 6077 char/sec
62% ppl: 392.516, speed: 6174 char/sec
72% ppl: 390.825, speed: 6246 char/sec
82% ppl: 389.389, speed: 6155 char/sec
92% ppl: 389.069, speed: 6045 char/sec
Epoch: 36 Train Perplexity: 388.843
Epoch: 37 Learning rate: 0.166
3% ppl: 397.144, speed: 7265 char/sec
13% ppl: 388.068, speed: 7253 char/sec
23% ppl: 393.321, speed: 7204 char/sec
33% ppl: 392.376, speed: 7044 char/sec
43% ppl: 393.071, speed: 6708 char/sec
52% ppl: 390.835, speed: 6531 char/sec
62% ppl: 388.354, speed: 6623 char/sec
72% ppl: 386.999, speed: 6548 char/sec
82% ppl: 385.507, speed: 6484 char/sec
92% ppl: 384.823, speed: 6551 char/sec
Epoch: 37 Train Perplexity: 384.504
Epoch: 38 Learning rate: 0.158
3% ppl: 391.009, speed: 7058 char/sec
13% ppl: 383.321, speed: 6928 char/sec
23% ppl: 389.161, speed: 7077 char/sec
33% ppl: 3

We can see the training process shown here. Observe that training loss keeps decreasing, which means that the model is actually learning. 

You can also continue training by calling the controller again. Try this later and see if the poems generated gets better over time.

## Clear previous output

Tensorflow will automatically load previous models if you specify a path for the `session`. However, that will be a problem if you change some parts of the model. e.g., change embedding size, LSTM size, or number of layers.

You will see something like 
```
INFO:tensorflow:Restoring parameters from logs/model.ckpt-4465
...
InvalidArgumentError: Assign requires shapes of both tensors to match.
```
Always remember to clear output directory if you are experimenting with different model structures!

In [15]:
!rm -R logs